### Main class to run the Algorithm

In [24]:
from Bellman_Ford import Graph
from Bellman_Ford import load_data


In [25]:


if __name__ == "__main__":
    # Load the dataset
    dataset_path = "data.csv"  # Replace with your dataset path
    num_airports, edges, airport_to_index = load_data(dataset_path)

    # Initialize the graph with the number of airports
    graph = Graph(num_airports)

    # Add edges from the dataset
    for source, destination, weight in edges:
        graph.add_edge(source, destination, weight)

    # Display the airport codes mapping
    index_to_airport = {idx: airport for airport, idx in airport_to_index.items()}
    print("\nAirport Codes Mapping:")
    for idx, airport in index_to_airport.items():
        print(f"{idx}: {airport}")

    # User input for source and destination airports
    source_code = input("\nEnter the source airport code (e.g., ORD): ")
    destination_code = input("Enter the destination airport code (e.g., SEA): ")

    if source_code not in airport_to_index or destination_code not in airport_to_index:
        print("Invalid airport code(s). Please check the mapping.")
    else:
        source_index = airport_to_index[source_code]
        destination_index = airport_to_index[destination_code]

        # Find the shortest paths using Bellman-Ford
        try:
            distances = graph.bellman_ford(source_index)

            # Display the shortest path to the destination
            shortest_distance = distances[destination_index]
            print(f"\nShortest distance from {source_code} to {destination_code}: {shortest_distance} miles")

            # Optional: Display distances to all airports
            print("\nDistances from source airport to all other airports:")
            for idx, distance in distances.items():
                print(f"{index_to_airport[idx]}: {distance} miles")

        except ValueError as e:
            print(e)


ValueError: invalid literal for int() with base 10: 'ORD'